# Source function variants for line-of-sight integration

In [ ]:
using Pkg
Pkg.activate()
Base.active_project()

In [ ]:
using SymBoltz, Plots, Unitful, UnitfulAstro
Plots.default(size = (1200, 500), margin = 8*Plots.mm, fmt = :png, dpi = 200, markersize = 2, markerstrokewidth = 0)

## Spherical Bessel functions

If one can compute $j_l(x)$ easily, one can also compute $j_l^\prime(x)$ and $j_l^″(x)$ easily from the identities
\begin{align}
j_l^\prime(x) &= \frac{l}{2l+1} j_{l-1}(x) - \frac{l+1}{2l+1} j_{l+1}(x), \\
x^2 j_l^″(x) &= \left( l(l+1) - x^2 \right) j_l(x) - 2x j_l^\prime(x).
\end{align}

* TODO: could set $j_l = j_l^\prime = 0$ and integrate last ODE while doing LOS integrals?

In [ ]:
jl(l, x) = SymBoltz.sphericalbesselj(l, x)
jl′(l, x) = l/(2l+1)*jl(l-1,x) - (l+1)/(2l+1)*jl(l+1,x)
jl″(l, x) = x == 0 ? jl″(l,1e-20) : (l*(l+1)/x^2 - 1)*jl(l,x) - 2/x*jl′(l,x)

l = 2
x = 0.0:0.1:100.0
plot(xlabel = "x", title = "l = $l")
plot!(x, jl.(l, x); label = "jₗ")
plot!(x, jl′.(l, x); label = "jₗ′")
plot!(x, jl″.(l, x); label = "jₗ″")

It is easier to capture the features of the functions with more derivatives. Note also these properties:
* $j_l(0) = 0$ for $l \geq 1$,
* $j_l^\prime(0) = 0$ for $l \geq 2$,
* $j_l^″(0) = 0$ for $l \geq 3$.

## Set up model

In [ ]:
M = ΛCDM(ν = nothing, h = nothing)
pars = SymBoltz.parameters_Planck18(M)
prob = CosmologyProblem(M, pars)
sol = solve(prob) # solve background only to get τ0

## Choosing time points

Take $\tau$ from background/thermodynamics ODE solution to automatically resolve recombination/reionization.

In [ ]:
τ0 = sol[M.τ0]
τ = sol[M.τ]
plot(τ, eachindex(τ); xlabel = "τ", ylabel = "N(≤τ)", label = "background τ")

## Line-of-sight integration by parts

Using integration by parts for any source function $S(\tau) \propto e^{-\kappa(\tau)}$, the line-of-sight integral can be written:
\begin{align}
I &= \int_0^{\tau_0} S^″(\tau) \cdot j_l(k(\tau_0-\tau)) \, \mathrm{d}\tau \qquad \text{(0 integrations by parts)} \\
  &= \Big[ S^\prime(\tau) \cdot j_l(k(\tau_0-\tau)) \Big]_{\tau=0}^{\tau=\tau_0} - \int_0^{\tau_0} (-k) S^\prime(\tau) \cdot j_l^\prime(k(\tau_0-\tau)) \, \mathrm{d}\tau \\
  &= \int_0^{\tau_0} (k S^\prime(\tau)) \cdot j_l^\prime(k(\tau_0-\tau)) \, \mathrm{d}\tau \qquad \text{(1 integrations by parts)} \\
  &= \Big[ k S(\tau) \cdot j_l^\prime(k(\tau_0-\tau)) \Big]_{\tau=0}^{\tau=\tau_0} - \int_0^{\tau_0} (-k) k S(\tau) \cdot j_l^″(k(\tau_0-\tau)) \, \mathrm{d}\tau \\
  &= \int_0^{\tau_0} (k^2 S(\tau)) \cdot j_l^″(k(\tau_0-\tau)) \, \mathrm{d}\tau \qquad \text{(2 integrations by parts)}
\end{align}

The boundary terms vanish for $l \geq 2$:
* at $\tau=0$, because $S(0) \propto e^{-\kappa(0)} = e^{-\infty} = 0$,
* at $\tau=\tau_0$, because $j_l(0) = j_l^\prime(0) = 0$ for $l \geq 2$.

These three line-of-sight integrals are equivalent.
But some may be easier to evaluate numerically than others.

In [ ]:
function plot_source_function_alternatives(Salts; l = 2, k = l/τ0, normalize = false, kwargs...)
    sol = solve(prob, k)

    xlabel = "τ"
    ylabel = "integrand"
    if normalize
        ylabel = "$ylabel / max(abs($ylabel))"
    end
    p = plot(; title = "l = $l, k = $(k/(l/τ0)) l/τ0", xlabel, ylabel)
    kχ = k*(τ0.-τ)
    kχ = max.(kχ, 0.0) # ensure ≥ 0
    for (n, Salt) in enumerate(Salts)
        S0 = length(Salt) >= 1 ? Salt[1] : 0
        S1 = length(Salt) >= 2 ? Salt[2] : 0
        S2 = length(Salt) >= 3 ? Salt[3] : 0
        S0 = isequal(S0, 0) ? zeros(length(τ)) : sol(k, τ, S0)
        S1 = isequal(S1, 0) ? zeros(length(τ)) : sol(k, τ, S1)
        S2 = isequal(S2, 0) ? zeros(length(τ)) : sol(k, τ, S2)
        y = @. S0 * jl(l, kχ) + S1 * jl′(l, kχ) + S2 * jl″(l, kχ)
        if normalize
            y = y / maximum(abs.(y))
        end
        plot!(τ, y; label = "alternative $n", kwargs...)
    end
    return p
end

The raw CMB temperature line-of-sight integral is
$$
\Theta_l(k,\tau_0) = \int_0^{\tau_0} \left[ v \left( \frac{\delta_\gamma}{4} + \Psi + \frac{\Pi_\gamma}{16} \right) + e^{-\kappa} \left( \Psi + \Phi \right)^\prime + \left( \frac{v u_b}{k} \right)^\prime + \left( \frac{3}{16} \frac{v \Pi_\gamma}{k^2} \right)^″ \right] j_l(k(\tau_0-\tau)) \, \mathrm{d}\tau.
$$

## 1. (Integrated) Sachs-Wolfe effect source function (2 alternatives)

\begin{align}
\Theta_l^\text{(I)SW}(k,\tau_0) &= \int_0^{\tau_0} \left[ v \left( \frac{\delta_\gamma}{4} + \Psi + \frac{\Pi_\gamma}{16} \right) + e^{-\kappa} (\Psi + \Phi)^\prime \right] \cdot j_l(k(\tau_0-\tau)) \, \mathrm{d\tau} \\
&= \int_0^{\tau_0} \left[ v \left( \frac{\delta_\gamma}{4} + \Psi + \frac{\Pi_\gamma}{16} \right) + \kappa^\prime e^{-\kappa} (\Psi + \Phi) \right] \cdot j_l(k(\tau_0-\tau)) \, \mathrm{d\tau} \\
&+ \int_0^{\tau_0} k (\Psi + \Phi) e^{-\kappa} \cdot j_l^\prime(k(\tau_0-\tau)) \, \mathrm{d}\tau \\
&= \int_0^{\tau_0} v \left( \frac{\delta_\gamma}{4} - \Phi + \frac{\Pi_\gamma}{16} \right) \cdot j_l(k(\tau_0-\tau)) \, \mathrm{d\tau} + \int_0^{\tau_0} k (\Psi + \Phi) e^{-\kappa} \cdot j_l^\prime(k(\tau_0-\tau)) \, \mathrm{d}\tau \\
&= \int_0^{\tau_0} \left[ v \left( \frac{\delta_\gamma}{4} + \Psi + \frac{\Pi_\gamma}{16} \right) + e^{-\kappa} (\Psi - \Phi)^\prime + 2 e^{-\kappa} \Phi^\prime \right] \cdot j_l(k(\tau_0-\tau)) \, \mathrm{d\tau} \\
&= \int_0^{\tau_0} \left[ v \left( \frac{\delta_\gamma}{4} + \Psi + \frac{\Pi_\gamma}{16} \right) + \kappa^\prime e^{-\kappa} (\Psi - \Phi) + 2 e^{-\kappa} \Phi^\prime \right] \cdot j_l(k(\tau_0-\tau)) \, \mathrm{d\tau} \\
&+ \int_0^{\tau_0} k (\Psi-\Phi) e^{-\kappa} \cdot j_l^\prime(k(\tau_0-\tau)) \, \mathrm{d}\tau \\
&= \int_0^{\tau_0} \left[ v \left( \frac{\delta_\gamma}{4} + \Phi + \frac{\Pi_\gamma}{16} \right) + 2 e^{-\kappa} \Phi^\prime \right] \cdot j_l(k(\tau_0-\tau)) \, \mathrm{d\tau} + \int_0^{\tau_0} k (\Psi-\Phi) e^{-\kappa} \cdot j_l^\prime(k(\tau_0-\tau)) \, \mathrm{d}\tau \\
\end{align}

In [ ]:
# Alternative 1:
S0 = M.b.rec.v * (M.γ.δ/4 + M.g.Ψ + M.γ.Π/16) + exp(-M.b.rec.κ) * (M.g.Ψ̇ - M.g.Φ̇)
S1 = 0
Salt1 = [S0, S1]

# Alternative 2:
S0 = M.b.rec.v * (M.γ.δ/4 + M.g.Φ + M.γ.Π/16) + 2 * exp(-M.b.rec.κ) * M.g.Φ̇
S1 = M.k * (M.g.Ψ - M.g.Φ) * exp(-M.b.rec.κ)
Salt2 = [S0, S1]

# Alternative 3:
S0 = M.b.rec.v * (M.γ.δ/4 - M.g.Φ + M.γ.Π/16)
S1 = M.k * exp(-M.b.rec.κ) * (M.g.Ψ + M.g.Φ)
Salt3 = [S0, S1]

plot_source_function_alternatives([Salt1, Salt2, Salt3]; l = 10, k = 1*l/τ0, marker = :circle)

## 2. Doppler effect source function (2 alternatives)

\begin{align}
\Theta_l^\text{Doppler}(k,\tau_0) &= \int_0^{\tau_0} \left( \frac{v \, u_b}{k} \right)^\prime \cdot j_l(k(\tau_0-\tau)) \, \mathrm{d}\tau \\
&= \int_0^{\tau_0} v \, u_b \cdot j_l^\prime(k(\tau_0-\tau)) \, \mathrm{d}\tau
\end{align}

In [ ]:
D = SymBoltz.D

# Alternative 2:
S0 = 0
S1 = M.b.rec.v * M.b.u
Salt2 = [S0, S1]

# Alternative 1:
S0 = SymBoltz.express_derivatives(D(S1) / M.k, prob)
S1 = 0
Salt1 = [S0, S1]

l = 50
k = 20*l/τ0
plot_source_function_alternatives([Salt1, Salt2]; l, k, marker = :circle)

## 3. Polarization source function (3 alternatives)

\begin{align}
\Theta_l^\text{polarization}(k,\tau_0) &= \int_0^{\tau_0} \left( \frac{3}{16} \frac{v \Pi}{k^2} \right)^″  \cdot j_l(k(\tau_0-\tau)) \, \mathrm{d}\tau \\
&= \int_0^{\tau_0} \left( \frac{3}{16} \frac{v \Pi}{k} \right)^\prime \cdot j_l^′(k(\tau_0-\tau)) \, \mathrm{d}\tau \\
&= \int_0^{\tau_0} \left( \frac{3}{16} v \Pi \right) \cdot j_l^″(k(\tau_0-\tau)) \, \mathrm{d}\tau \\
\end{align}

In [ ]:
Salt1 = [M.ST0_polarization, 0, 0]
Salt2 = [0, M.ST1_polarization, 0]
Salt3 = [0, 0, M.ST2_polarization]
plot_source_function_alternatives([Salt1, Salt2, Salt3]; l = 5, marker = :circle, normalize = true)